In [1]:
import os

In [26]:
os.chdir("../")

In [25]:
%pwd

'c:\\Users\\zeyne\\Containerized-Bank-Customer-Churn-Prediction\\research'

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [28]:
@dataclass (frozen = True)
class ModelTrainerConfig:
    root_dir : Path
    train_data_path : Path
    test_data_path : Path
    model : str
    target_column : str 
    parameters : dict[str,any]
    model_name : str

In [29]:
from src.Containerized_Bank_Customed_Churn_Prediction.constants import *
from src.Containerized_Bank_Customed_Churn_Prediction.utils.common import read_yaml,create_directories

In [43]:
class ConfigurationManager:
 
 def __init__(self, config_filepath = CONFIG_FILE_PATH ,params_filepath = PARAMS_FILE_PATH, schema_filepath = SCHEMA_FILE_PATH):

    self.config = read_yaml(config_filepath)
    self.params = read_yaml(params_filepath)
    self.schema = read_yaml(schema_filepath)

    create_directories([self.config.artifacts_root])

    

 def get_model_trainer_config(self) ->ModelTrainerConfig:
     config = self.config.model_trainer
     schema = self.schema.TARGET_COLUMN
     params = self.params


     model_name = params["model_name"]
     model_trainer_config = ModelTrainerConfig(
        root_dir = config.root_dir,
        train_data_path = config.train_data_path,
        test_data_path = config.test_data_path,
        model = config.model,
        target_column = schema.name,
        parameters = params[model_name],
        model_name = model_name
     )

     return model_trainer_config

In [12]:
import os 
from Containerized_Bank_Customed_Churn_Prediction import logger
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
import joblib

In [56]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config = config
        
    def train(self):

        preprocessor = joblib.load("artifacts/data_transformation/preprocessor.pkl")
        
        parameters=self.config.parameters
        model_class = RandomForestClassifier

        full_pipe =Pipeline(steps=[
                ('preprocessor',preprocessor),
                ('model', model_class(**parameters))
            ])



        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop(self.config.target_column,axis = 1)
        test_x = test_data.drop(self.config.target_column,axis = 1)

        train_y = train_data[[self.config.target_column]]
        test_y =  test_data[[self.config.target_column]]


        full_pipe.fit(train_x, train_y.values.ravel())

        joblib.dump(full_pipe, os.path.join(self.config.root_dir,self.config.model_name))



        

In [57]:
try:
    config = ConfigurationManager()
    model_tariner_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_tariner_config)
    model_trainer.train()

    
except Exception as e:
    raise e

[2025-08-06 23:27:55,944:INFO: common: yaml file:config\config.yaml loaded successfully]
[2025-08-06 23:27:55,945:INFO: common: yaml file:params.yaml loaded successfully]
[2025-08-06 23:27:55,950:INFO: common: yaml file:schema.yaml loaded successfully]
[2025-08-06 23:27:55,952:INFO: common: created directory at:artifacts]
